In [1]:
# Load the model that we created in Part 2
from gensim.models import Word2Vec
model = Word2Vec.load("models\\300features_40minwords_10context")

In [2]:
import pandas as pd

# Read training data
train = pd.read_csv("data\\labeledTrainData.tsv", header=0, 
    delimiter="\t", quoting=3 )

test = pd.read_csv("data\\testData.tsv", header=0, delimiter="\t", quoting=3)
unlabelled_train = pd.read_csv("data\\unlabeledTrainData.tsv", header=0, 
    delimiter="\t", quoting=3)

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labelled train reviews, %d labelled test reviews, " \
 "and %d unlabelled reviews\n" % (train["review"].size,  \
 test["review"].size, unlabelled_train["review"].size ))

Read 25000 labelled train reviews, 25000 labelled test reviews, and 50000 unlabelled reviews



In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# syn0 property contains the feature vectors.
# We've got about 17k words overall with 300 features to describe each one.
print(model.syn0.shape)
print(model['flower'])

(16490, 300)
[ -6.33130446e-02  -9.51148346e-02  -1.13297123e-02   2.06500944e-02
   7.69971497e-03   3.90268639e-02  -2.31053215e-03   5.74429892e-02
  -7.89635256e-02  -8.16961229e-02  -4.67109419e-02  -2.64932867e-02
  -3.51126157e-02   2.89048813e-02   4.62030284e-02  -3.96941677e-02
   7.13620633e-02   1.02953045e-02   5.50104231e-02   8.82932693e-02
  -5.12857474e-02  -6.33965358e-02   9.97599661e-02  -8.15247477e-04
   1.72204655e-02   6.61159446e-03   5.32784984e-02   4.57147136e-02
  -6.43707365e-02   7.38542452e-02   6.92851003e-03  -7.33929053e-02
   5.16038295e-03   7.72529915e-02  -2.53960341e-02   4.63377638e-03
   8.82617850e-03  -3.54095275e-04   8.12259167e-02  -6.88227639e-02
  -4.25982922e-02   2.36869045e-02  -3.61694470e-02  -1.70206651e-02
   1.36775151e-02  -9.27254185e-02   2.43001468e-02  -7.25661069e-02
   1.55967183e-03   2.96756774e-02   6.78326264e-02  -1.84335455e-01
  -1.54907286e-01  -8.04380104e-02   5.98481931e-02  -9.95170400e-02
  -9.91423801e-02  -4

In [5]:
# this list contains all the words in the model vocabulary
model.index2word

['the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'i',
 'this',
 'that',
 's',
 'was',
 'as',
 'with',
 'for',
 'movie',
 'but',
 'film',
 'you',
 't',
 'on',
 'not',
 'he',
 'are',
 'his',
 'have',
 'be',
 'one',
 'all',
 'at',
 'they',
 'by',
 'who',
 'an',
 'from',
 'so',
 'like',
 'there',
 'her',
 'or',
 'just',
 'about',
 'out',
 'has',
 'if',
 'what',
 'some',
 'good',
 'can',
 'more',
 'when',
 'very',
 'she',
 'up',
 'no',
 'time',
 'even',
 'would',
 'my',
 'which',
 'their',
 'story',
 'only',
 'really',
 'see',
 'had',
 'were',
 'well',
 'we',
 'me',
 'than',
 'much',
 'bad',
 'get',
 'been',
 'people',
 'also',
 'into',
 'do',
 'great',
 'other',
 'will',
 'first',
 'because',
 'him',
 'how',
 'most',
 'don',
 'them',
 'made',
 'its',
 'make',
 'then',
 'way',
 'could',
 'too',
 'movies',
 'after',
 'any',
 'characters',
 'character',
 'think',
 'films',
 'two',
 'watch',
 'being',
 'many',
 'plot',
 'seen',
 'never',
 'where',
 'love',
 'life',
 'little',
 'acting

In [6]:
import numpy as np  # Make sure that numpy is imported

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocabulary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [7]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.
num_features = 300
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, \
        remove_stopwords=True))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, \
        remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

C:\Miniconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Miniconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


Review 0 of 25000


Review 1000 of 25000
Review 2000 of 25000


Review 3000 of 25000
Review 4000 of 25000


Review 5000 of 25000
Review 6000 of 25000


...

Review 18000 of 25000


Review 19000 of 25000


Review 20000 of 25000


Review 21000 of 25000
Review 22000 of 25000


Review 23000 of 25000
Review 24000 of 25000


In [8]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100, oob_score=True )

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainDataVecs, train["sentiment"])

# Test & extract results 
result = forest.predict(testDataVecs)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("Word2Vec_AverageVectors.csv", index=False, quoting=3)

Fitting a random forest to labeled training data...


In [9]:
print(forest.oob_score_)

0.82904


In [10]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans(n_clusters = num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  659.7517893314362 seconds.


In [13]:
# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

In [18]:
# For the first 10 clusters
for cluster in range(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(0,len(word_centroid_map.values())):
        if(list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0


['fischer']

Cluster 1


['symbolism', 'literal', 'metaphorical', 'symbolic', 'rooted', 'warped', 'hazy', 'contradiction', 'skewed']

Cluster 2


['beating']

Cluster 3


['kiki']

Cluster 4


...

['liz']

Cluster 6


['fare', 'comedies', 'movies', 'films', 'slashers', 'thrillers', 'pics', 'flicks', 'dramas']

Cluster 7


['hapless', 'headed', 'nutty', 'witted', 'heavies', 'bumbling', 'misfit']

Cluster 8


['mexicans', 'turks', 'vikings', 'brits', 'russians', 'europeans']

Cluster 9


['coltrane', 'magnum']


In [20]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [21]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# Repeat for test reviews 
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [ ]:
# Fit a random forest and extract predictions 
forest = RandomForestClassifier(n_estimators = 100)

# Fitting the forest may take a few minutes
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("BagOfCentroids.csv", index=False, quoting=3)

In [ ]:
# Now let's try plain old logistic regression
from sklearn.linear_model import LogisticRegression as LR
model = LR()
lr = model.fit(train_centroids,train["sentiment"])
result = model.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv("BagOfCentroidsLogisticRegression.csv", index=False, quoting=3)